## 递归神经网络进阶

<img src="data/logo.png" alt="Drawing" style="width: 300px;"/>

这个项目中我们会包含许多RNN的进阶知识内容。

1. 条件隐藏状态
2. 字符级嵌入
3. 编码器和解码器
4. 注意机制
5. 实践

## 配置

In [1]:
!pip install torch torchvision --upgrade -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Requirement already up-to-date: torch in /opt/conda/lib/python3.5/site-packages (1.0.0)
Requirement already up-to-date: torchvision in /opt/conda/lib/python3.5/site-packages (0.2.1)
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import os
from argparse import Namespace
import collections
import copy
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import torch

In [3]:
# Set Numpy and PyTorch seeds
def set_seeds(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)
        
# Creating directories
def create_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

In [4]:
# Arguments
args = Namespace(
    seed=1234,
    cuda=True,
    batch_size=4,
    condition_vocab_size=3, # vocabulary for condition possibilities
    embedding_dim=100,
    rnn_hidden_dim=100,
    hidden_dim=100,
    num_layers=1,
    bidirectional=False,
)

# Set seeds
set_seeds(seed=args.seed, cuda=args.cuda)

# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False
args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

Using CUDA: True


# 条件递归神经网络
给RNN加上条件判断实际上是在为正确的预测添加附加信息。我们可以将这些信息编码(嵌入)然后将它和序列输入数据一起穿进模型。依旧举上个项目中文本分类的例子，假设每篇文章的出版商(NYTimes， ESPN 等)已知。我们就可以用这个信息来帮助预测(比如ESPN发行的新闻更可能是体育新闻)。
**注意**: 如果条件信息对序列中每个输入都不一样，那只需将它和每个时间步的数据联系起来就好了






**第一步**： 把初始隐藏状态替换为编码后的条件信息，保证它的大小和RNN的隐藏状态大小一样

<img src="data/conditioned_rnn1.png" alt="Drawing" style="width: 600px;"/>


In [5]:
import torch.nn as nn
import torch.nn.functional as F

In [6]:
# 条件
condition = torch.LongTensor([0, 2, 1, 2]) # 批大小4， 词汇表大小3
condition_embeddings = nn.Embedding(
    embedding_dim=args.embedding_dim, # 应该和RNN隐藏维一样
    num_embeddings=args.condition_vocab_size) # 条件各异

# 初始化隐藏状态
num_directions = 1
if args.bidirectional:
    num_directions = 2
    
# If using multiple layers and directions, the hidden state needs to match that size
# 如果使用了多层或者多向网络， 隐藏状态需要匹配大小
hidden_t = condition_embeddings(condition).unsqueeze(0).repeat(
    args.num_layers * num_directions, 1, 1).to(args.device) # initial state to RNN
print (hidden_t.size())

# 传入rnn
# y_out, _ = self.rnn(x_embedded, hidden_t)


torch.Size([1, 4, 100])



**第二步**: 将编码过的信息和每一时间步的隐藏状态关联，注意这里**千万不要**替换隐藏状态，RNN需要它来进行训练

<img src="data/conditioned_rnn2.png" alt="Drawing" style="width: 600px;"/>


In [7]:
# 初始化隐藏状态
hidden_t = torch.zeros((args.num_layers * num_directions, args.batch_size, args.rnn_hidden_dim))
print (hidden_t.size())

torch.Size([1, 4, 100])


In [8]:
def concat_condition(condition_embeddings, condition, hidden_t, num_layers, num_directions):
    condition_t = condition_embeddings(condition).unsqueeze(0).repeat(
        num_layers * num_directions, 1, 1)
    hidden_t = torch.cat([hidden_t, condition_t], 2)
    return hidden_t

In [9]:
# 循环输入
hiddens = []
seq_size = 1
for t in range(seq_size):
    hidden_t = concat_condition(condition_embeddings, condition, hidden_t, 
                                args.num_layers, num_directions).to(args.device)
    print (hidden_t.size())
    
    # 传入rnn
    # hidden_t = rnn_cell(x_in[t], hidden_t)
    ...

torch.Size([1, 4, 200])


## 字符级嵌入

卷积操作可能会有句子中包含用字符表示的单词 |  ~$\in \mathbb{R}^{NXSXWXE}~$ 而输出是每个词的嵌入(根据具体的卷积操作)。

**词嵌入**: 找到相邻token中的暂时关系，这些暂时关系暗示了某些词语代表了相似的意义。比如 "New Jersey" 和 "NJ" 和 "Garden State" 有相似意思，他们都指新泽信这个地方。

**字符嵌入**: 创建词语在字符级别的关联。比如 "toy" 和 "toys" 意思相近。

<img src="data/char_embeddings.png" alt="Drawing" style="width: 600px;"/>


In [10]:
# 参数
args = Namespace(
    seed=1234,
    cuda=False,
    shuffle=True,
    batch_size=64,
    vocab_size=20, # 词汇表
    seq_size=10, # 句子的最长长度
    word_size=15, # 词语的最长长度
    embedding_dim=100,
    num_filters=100, # 过滤器大小
)

In [11]:
class Model(nn.Module):
    def __init__(self, embedding_dim, num_embeddings, num_input_channels, 
                 num_output_channels, padding_idx):
        super(Model, self).__init__()
        
        # 字符嵌入
        self.embeddings = nn.Embedding(embedding_dim=embedding_dim,
                                       num_embeddings=num_embeddings,
                                       padding_idx=padding_idx)
        
        # 卷积权重
        self.conv = nn.ModuleList([nn.Conv1d(num_input_channels, num_output_channels, 
                                             kernel_size=f) for f in [2,3,4]])

    def forward(self, x, channel_first=False, apply_softmax=False):
        
        # x: (N, seq_len, word_len)
        input_shape = x.size()
        batch_size, seq_len, word_len = input_shape
        x = x.view(-1, word_len) # (N*seq_len, word_len)
        
        # 嵌入
        x = self.embeddings(x) # (N*seq_len, word_len, embedding_dim)
        
        # 重新排布输入各维位置 (N, embedding_dim, word_len)
        if not channel_first:
            x = x.transpose(1, 2)
        
        # 卷积
        z = [F.relu(conv(x)) for conv in self.conv]
        
        # 池化
        z = [F.max_pool1d(zz, zz.size(2)).squeeze(2) for zz in z] 
        z = [zz.view(batch_size, seq_len, -1) for zz in z] # (N, seq_len, embedding_dim)
        
        # 拼接或者字符级嵌入
        z = torch.cat(z, 2) 
        
        return z

In [12]:
# 输入
input_size = (args.batch_size, args.seq_size, args.word_size)
x_in = torch.randint(low=0, high=args.vocab_size, size=input_size).long()
print (x_in.size())

torch.Size([64, 10, 15])


In [13]:
# 初始化字符级嵌入
model = Model(embedding_dim=args.embedding_dim, 
              num_embeddings=args.vocab_size, 
              num_input_channels=args.embedding_dim, 
              num_output_channels=args.num_filters,
              padding_idx=0)
print (model.named_modules)

<bound method Module.named_modules of Model(
  (embeddings): Embedding(20, 100, padding_idx=0)
  (conv): ModuleList(
    (0): Conv1d(100, 100, kernel_size=(2,), stride=(1,))
    (1): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
    (2): Conv1d(100, 100, kernel_size=(4,), stride=(1,))
  )
)>


In [14]:
# 向前传播得到字符级嵌入
z = model(x_in)
print (z.size())

torch.Size([64, 10, 300])


字符嵌入有以下几种用法:

1. 将字符嵌入和词嵌入拼接，然后传入RNN
2. 直接传入RNN

## 编码器和解码器

到现在为止，我们都在使用RNN来对序列化输入进行 `编码` 得到隐藏状态。然后使用这些隐藏状态去对预测值 `解码`。我们使用的编码器就是RNN而解码器只是几层全连接层接着一层softmax(用于分类)。但其实编码器和解码器都可以有其他的结构。比如解码器也可以是一个RNN。

In [15]:
# 参数
args = Namespace(
    batch_size=64,
    embedding_dim=100,
    rnn_hidden_dim=100,
    hidden_dim=100,
    num_layers=1,
    bidirectional=False,
    dropout=0.1,
)

In [16]:
class Encoder(nn.Module):
    def __init__(self, embedding_dim, num_embeddings, rnn_hidden_dim, 
                 num_layers, bidirectional, padding_idx=0):
        super(Encoder, self).__init__()
        
        # 嵌入
        self.word_embeddings = nn.Embedding(embedding_dim=embedding_dim,
                                            num_embeddings=num_embeddings,
                                            padding_idx=padding_idx)
        
        # GRU权重
        self.gru = nn.GRU(input_size=embedding_dim, hidden_size=rnn_hidden_dim, 
                          num_layers=num_layers, batch_first=True, 
                          bidirectional=bidirectional)

    def forward(self, x_in, x_lengths):
        
        # 词级嵌入
        z_word = self.word_embeddings(x_in)
   
        # 传入RNN
        out, h_n = self.gru(z)
        
        # 获取最新的相关隐藏状态
        out = gather_last_relevant_hidden(out, x_lengths)
        
        return out

In [17]:
class Decoder(nn.Module):
    def __init__(self, rnn_hidden_dim, hidden_dim, output_dim, dropout_p):
        super(Decoder, self).__init__()
        
        # 全连接权重
        self.dropout = nn.Dropout(dropout_p)
        self.fc1 = nn.Linear(rnn_hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, encoder_output, apply_softmax=False):
        
        # 全连接层
        z = self.dropout(encoder_output)
        z = self.fc1(z)
        z = self.dropout(z)
        y_pred = self.fc2(z)

        if apply_softmax:
            y_pred = F.softmax(y_pred, dim=1)
        return y_pred

In [18]:
class Model(nn.Module):
    def __init__(self, embedding_dim, num_embeddings, rnn_hidden_dim, 
                 hidden_dim, num_layers, bidirectional, output_dim, dropout_p, 
                 padding_idx=0):
        super(Model, self).__init__()
        self.encoder = Encoder(embedding_dim, num_embeddings, rnn_hidden_dim, 
                               num_layers, bidirectional, padding_idx=0)
        self.decoder = Decoder(rnn_hidden_dim, hidden_dim, output_dim, dropout_p)
        
    def forward(self, x_in, x_lengths, apply_softmax=False):
        encoder_outputs = self.encoder(x_in, x_lengths)
        y_pred = self.decoder(encoder_outputs, apply_softmax)
        return y_pred

In [19]:
model = Model(embedding_dim=args.embedding_dim, num_embeddings=1000, 
              rnn_hidden_dim=args.rnn_hidden_dim, hidden_dim=args.hidden_dim, 
              num_layers=args.num_layers, bidirectional=args.bidirectional, 
              output_dim=4, dropout_p=args.dropout, padding_idx=0)
print (model.named_parameters)

<bound method Module.named_parameters of Model(
  (encoder): Encoder(
    (word_embeddings): Embedding(1000, 100, padding_idx=0)
    (gru): GRU(100, 100, batch_first=True)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.1)
    (fc1): Linear(in_features=100, out_features=100, bias=True)
    (fc2): Linear(in_features=100, out_features=4, bias=True)
  )
)>


## 注意力机制 (Attentional mechanisms)

我们知道RNN在处理输入序列的时候，在每个时间步被处理的是那一时刻的输入和隐藏状态。在大部分情况下，如果可以访问所有输入然后在每一个时间步只关注某一些被选择出的值时比较优秀的实践。比如在机器翻译时，最好可以访问被翻译内容中的所有值，因为在翻译的过程中，结果和原句并不是严格词与词一一对应的。


<img src="data/attention1.jpg" alt="Drawing" style="width: 600px;"/>

注意力机制听起来可能令人疑惑，所以我们来看看每一个时间步都发生了什么。
在时间步 j, 模型已经处理了 ~$x_0, x_1, x_2, ..., x_j~$, 并且生成了隐藏状态 ~$h_0, h_1, h_2, ..., h_j~$。 主要思想其实是，在做预测时用到所有已被处理的隐藏状态，而不是最近的那一个。有几种方法可以达成这样的效果。

**软注意 soft attention** 是学习并获得一个浮点数向量(概率)，然后和隐藏状态相乘得到 表示上下文的向量(context vector)


比如. [0.1, 0.3, 0.1, 0.4, 0.1]


**硬注意 hard attention** 是学习并获得一个进制向量, 然后和隐藏状态相乘得到 表示上下文的向量(context vector)
比如. [0, 0, 0, 1, 0]

这里我们会主要介绍 soft attention 因为它的使用其实更加广泛，并且对  每个隐藏状态对最终预测的影响  作简单的可视化。也就是它的可解释性更好

<img src="data/attention2.jpg" alt="Drawing" style="width: 600px;"/>

我们将在下面的文本分类任务中使用注意力机制。

## 使用RNN进行文本分类

我们会完成和前个项目一样的文本分类任务，但是会使用注意力机制来解读模型。

**为什么不用机器翻译模型(machine translation)?** 一般来说在解释注意力机制时大家都会直接拿机器翻译模型做样例，但其实这样做并不是很实际。你并不能经常看到用一个序列来生成另一个序列的实际情景。所以我们会使用文本分类的例子来解读那些输入词条对分类有更强的影响力。

### Set up

In [20]:
from argparse import Namespace
import collections
import copy
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import torch

In [21]:
def set_seeds(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)
        
# Creating directories
def create_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

In [22]:
args = Namespace(
    seed=1234,
    cuda=True,
    shuffle=True,
    data_file="data/news.csv",
    split_data_file="data/split_news.csv",
    vectorizer_file="vectorizer.json",
    model_state_file="model.pth",
    save_dir="news",
    train_size=0.7,
    val_size=0.15,
    test_size=0.15,
    pretrained_embeddings=None,
    cutoff=25,
    num_epochs=5,
    early_stopping_criteria=5,
    learning_rate=1e-3,
    batch_size=128,
    embedding_dim=100,
    kernels=[3,5],
    num_filters=100,
    rnn_hidden_dim=128,
    hidden_dim=200,
    num_layers=1,
    bidirectional=False,
    dropout_p=0.25,
)

# Set seeds
set_seeds(seed=args.seed, cuda=args.cuda)

# Create save dir
create_dirs(args.save_dir)

# Expand filepaths
args.vectorizer_file = os.path.join(args.save_dir, args.vectorizer_file)
args.model_state_file = os.path.join(args.save_dir, args.model_state_file)

# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False
args.device = torch.device("cuda" if args.cuda else "cpu")
print("Using CUDA: {}".format(args.cuda))

Using CUDA: True


### 数据

In [23]:
import urllib

In [24]:
url = "https://raw.githubusercontent.com/GokuMohandas/practicalAI/master/data/news.csv"
response = urllib.request.urlopen(url)
html = response.read()
with open(args.data_file, 'wb') as fp:
    fp.write(html)

In [25]:
df = pd.read_csv(args.data_file, header=0)
df.head()

,category,title
0,Business,Wall St. Bears Claw Back Into the Black (Reuters)
1,Business,Carlyle Looks Toward Commercial Aerospace (Reu...
2,Business,Oil and Economy Cloud Stocks' Outlook (Reuters)
3,Business,Iraq Halts Oil Exports from Main Southern Pipe...
4,Business,"Oil prices soar to all-time record, posing new..."


In [26]:
by_category = collections.defaultdict(list)
for _, row in df.iterrows():
    by_category[row.category].append(row.to_dict())
for category in by_category:
    print ("{0}: {1}".format(category, len(by_category[category])))

Sports: 30000
World: 30000
Business: 30000
Sci/Tech: 30000


In [27]:
final_list = []
for _, item_list in sorted(by_category.items()):
    if args.shuffle:
        np.random.shuffle(item_list)
    n = len(item_list)
    n_train = int(args.train_size*n)
    n_val = int(args.val_size*n)
    n_test = int(args.test_size*n)

  # Give data point a split attribute
    for item in item_list[:n_train]:
        item['split'] = 'train'
    for item in item_list[n_train:n_train+n_val]:
        item['split'] = 'val'
    for item in item_list[n_train+n_val:]:
        item['split'] = 'test'  

    # Add to final list
    final_list.extend(item_list)

In [28]:
split_df = pd.DataFrame(final_list)
split_df["split"].value_counts()

train    84000
test     18000
val      18000
Name: split, dtype: int64

In [29]:
def preprocess_text(text):
    text = ' '.join(word.lower() for word in text.split(" "))
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    text = text.strip()
    return text
    
split_df.title = split_df.title.apply(preprocess_text)

In [30]:
split_df.to_csv(args.split_data_file, index=False)
split_df.head()

,category,split,title
0,Business,train,general electric posts higher rd quarter profit
1,Business,train,lilly to eliminate up to us jobs
2,Business,train,s amp p lowers america west outlook to negative
3,Business,train,does rand walk the talk on labor policy ?
4,Business,train,housekeeper advocates for changes


### 词汇表

In [31]:
class Vocabulary(object):
    def __init__(self, token_to_idx=None):

        # Token to index
        if token_to_idx is None:
            token_to_idx = {}
        self.token_to_idx = token_to_idx

        # Index to token
        self.idx_to_token = {idx: token \
                             for token, idx in self.token_to_idx.items()}

    def to_serializable(self):
        return {'token_to_idx': self.token_to_idx}

    @classmethod
    def from_serializable(cls, contents):
        return cls(**contents)

    def add_token(self, token):
        if token in self.token_to_idx:
            index = self.token_to_idx[token]
        else:
            index = len(self.token_to_idx)
            self.token_to_idx[token] = index
            self.idx_to_token[index] = token
        return index

    def add_tokens(self, tokens):
        return [self.add_token[token] for token in tokens]

    def lookup_token(self, token):
        return self.token_to_idx[token]

    def lookup_index(self, index):
        if index not in self.idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self.idx_to_token[index]

    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)

    def __len__(self):
        return len(self.token_to_idx)

In [32]:
# Vocabulary instance
category_vocab = Vocabulary()
for index, row in df.iterrows():
    category_vocab.add_token(row.category)
print (category_vocab) # __str__
print (len(category_vocab)) # __len__
index = category_vocab.lookup_token("Business")
print (index)
print (category_vocab.lookup_index(index))

<Vocabulary(size=4)>
4
0
Business


### 序列词汇表
接下来我们将为文章标题创建词汇表类，它由一系列词条构成。

In [33]:
from collections import Counter
import string

In [34]:
class SequenceVocabulary(Vocabulary):
    def __init__(self, token_to_idx=None, unk_token="<UNK>",
                 mask_token="<MASK>", begin_seq_token="<BEGIN>",
                 end_seq_token="<END>"):

        super(SequenceVocabulary, self).__init__(token_to_idx)

        self.mask_token = mask_token
        self.unk_token = unk_token
        self.begin_seq_token = begin_seq_token
        self.end_seq_token = end_seq_token

        self.mask_index = self.add_token(self.mask_token)
        self.unk_index = self.add_token(self.unk_token)
        self.begin_seq_index = self.add_token(self.begin_seq_token)
        self.end_seq_index = self.add_token(self.end_seq_token)
        
        # Index to token
        self.idx_to_token = {idx: token \
                             for token, idx in self.token_to_idx.items()}

    def to_serializable(self):
        contents = super(SequenceVocabulary, self).to_serializable()
        contents.update({'unk_token': self.unk_token,
                         'mask_token': self.mask_token,
                         'begin_seq_token': self.begin_seq_token,
                         'end_seq_token': self.end_seq_token})
        return contents

    def lookup_token(self, token):
        return self.token_to_idx.get(token, self.unk_index)
    
    def lookup_index(self, index):
        if index not in self.idx_to_token:
            raise KeyError("the index (%d) is not in the SequenceVocabulary" % index)
        return self.idx_to_token[index]
    
    def __str__(self):
        return "<SequenceVocabulary(size=%d)>" % len(self.token_to_idx)

    def __len__(self):
        return len(self.token_to_idx)


In [35]:
# Get word counts
word_counts = Counter()
for title in split_df.title:
    for token in title.split(" "):
        if token not in string.punctuation:
            word_counts[token] += 1

# Create SequenceVocabulary instance
title_word_vocab = SequenceVocabulary()
for word, word_count in word_counts.items():
    if word_count >= args.cutoff:
        title_word_vocab.add_token(word)
print (title_word_vocab) # __str__
print (len(title_word_vocab)) # __len__
index = title_word_vocab.lookup_token("general")
print (index)
print (title_word_vocab.lookup_index(index))

<SequenceVocabulary(size=4400)>
4400
1092
general



我们同时会创建SequenceVocabulary实例，它会在字符层级处理输入。

In [36]:
# Create SequenceVocabulary instance
title_char_vocab = SequenceVocabulary()
for title in split_df.title:
    for token in title:
        title_char_vocab.add_token(token)
print (title_char_vocab) # __str__
print (len(title_char_vocab)) # __len__
index = title_char_vocab.lookup_token("g")
print (index)
print (title_char_vocab.lookup_index(index))

<SequenceVocabulary(size=35)>
35
4
g


### 向量化
在向量化中这次我们会引入新的操作: 计算输入序列的长度。我们会在为每个输入序列提取最新的相关隐藏状态时用到它

In [38]:
# Vectorizer instance
vectorizer = NewsVectorizer.from_dataframe(split_df, cutoff=args.cutoff)
print (vectorizer.title_word_vocab)
print (vectorizer.title_char_vocab)
print (vectorizer.category_vocab)
word_vector, char_vector, title_length = vectorizer.vectorize(preprocess_text(
    "Roger Federer wins the Wimbledon tennis tournament."))
print ("word_vector:", np.shape(word_vector))
print ("char_vector:", np.shape(char_vector))
print ("title_length:", title_length)
print (word_vector)
print (char_vector)
print (vectorizer.unvectorize_word_vector(word_vector))
print (vectorizer.unvectorize_char_vector(char_vector))

<SequenceVocabulary(size=4404)>
<SequenceVocabulary(size=35)>
<Vocabulary(size=4)>
word_vector: (10,)
char_vector: (10, 10)
title_length: 10
[   2    1 2843 3993 2552    1 3319  410 4236    3]
[[ 0  0  0  0  0  0  0  0  0  0]
 [ 7 15  4  5  7  0  0  0  0  0]
 [21  5 18  5  7  5  7  0  0  0]
 [26 13  6 16  0  0  0  0  0  0]
 [12 17  5  0  0  0  0  0  0  0]
 [26 13 23 25  9  5 18 15  6  0]
 [12  5  6  6 13 16  0  0  0  0]
 [12 15 20  7  6  8 23  5  6 12]
 [30  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]]
<BEGIN> <UNK> federer wins the <UNK> tennis tournament . <END>
 roger federer wins the wimbledon tennis tournament .  


### 数据

In [39]:
from torch.utils.data import Dataset, DataLoader

In [40]:
class NewsDataset(Dataset):
    def __init__(self, df, vectorizer):
        self.df = df
        self.vectorizer = vectorizer

        # Data splits
        self.train_df = self.df[self.df.split=='train']
        self.train_size = len(self.train_df)
        self.val_df = self.df[self.df.split=='val']
        self.val_size = len(self.val_df)
        self.test_df = self.df[self.df.split=='test']
        self.test_size = len(self.test_df)
        self.lookup_dict = {'train': (self.train_df, self.train_size), 
                            'val': (self.val_df, self.val_size),
                            'test': (self.test_df, self.test_size)}
        self.set_split('train')

        # Class weights (for imbalances)
        class_counts = df.category.value_counts().to_dict()
        def sort_key(item):
            return self.vectorizer.category_vocab.lookup_token(item[0])
        sorted_counts = sorted(class_counts.items(), key=sort_key)
        frequencies = [count for _, count in sorted_counts]
        self.class_weights = 1.0 / torch.tensor(frequencies, dtype=torch.float32)

    @classmethod
    def load_dataset_and_make_vectorizer(cls, split_data_file, cutoff):
        df = pd.read_csv(split_data_file, header=0)
        train_df = df[df.split=='train']
        return cls(df, NewsVectorizer.from_dataframe(train_df, cutoff))

    @classmethod
    def load_dataset_and_load_vectorizer(cls, split_data_file, vectorizer_filepath):
        df = pd.read_csv(split_data_file, header=0)
        vectorizer = cls.load_vectorizer_only(vectorizer_filepath)
        return cls(df, vectorizer)

    def load_vectorizer_only(vectorizer_filepath):
        with open(vectorizer_filepath) as fp:
            return NewsVectorizer.from_serializable(json.load(fp))

    def save_vectorizer(self, vectorizer_filepath):
        with open(vectorizer_filepath, "w") as fp:
            json.dump(self.vectorizer.to_serializable(), fp)

    def set_split(self, split="train"):
        self.target_split = split
        self.target_df, self.target_size = self.lookup_dict[split]

    def __str__(self):
        return "<Dataset(split={0}, size={1})".format(
            self.target_split, self.target_size)

    def __len__(self):
        return self.target_size

    def __getitem__(self, index):
        row = self.target_df.iloc[index]
        title_word_vector, title_char_vector, title_length = \
            self.vectorizer.vectorize(row.title)
        category_index = self.vectorizer.category_vocab.lookup_token(row.category)
        return {'title_word_vector': title_word_vector, 
                'title_char_vector': title_char_vector, 
                'title_length': title_length, 
                'category': category_index}

    def get_num_batches(self, batch_size):
        return len(self) // batch_size

    def generate_batches(self, batch_size, collate_fn, shuffle=True, 
                         drop_last=False, device="cpu"):
        dataloader = DataLoader(dataset=self, batch_size=batch_size,
                                collate_fn=collate_fn, shuffle=shuffle, 
                                drop_last=drop_last)
        for data_dict in dataloader:
            out_data_dict = {}
            for name, tensor in data_dict.items():
                out_data_dict[name] = data_dict[name].to(device)
            yield out_data_dict

In [41]:
# Dataset instance
dataset = NewsDataset.load_dataset_and_make_vectorizer(args.split_data_file,
                                                       args.cutoff)
print (dataset) # __str__
input_ = dataset[10] # __getitem__
print (input_['title_word_vector'])
print (input_['title_char_vector'])
print (input_['title_length'])
print (input_['category'])
print (dataset.vectorizer.unvectorize_word_vector(input_['title_word_vector']))
print (dataset.vectorizer.unvectorize_char_vector(input_['title_char_vector']))
print (dataset.class_weights)

<Dataset(split=train, size=84000)
[   2  579    1 1645 2973 2867  522    3]
[[ 0  0  0  0  0  0  0  0  0  0]
 [18  5  9 12  8  0  0  0  0  0]
 [18 15 18  4  5 16  0  0  0  0]
 [25  8  6 27  7 20 14 12 11 22]
 [26 13 12 17  0  0  0  0  0  0]
 [ 9  8 25 15  7  0  0  0  0  0]
 [18  5  8  9  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0]]
8
0
<BEGIN> delta <UNK> bankruptcy with labor deal <END>
 delta dodges bankruptcy with labor deal  
tensor([3.3333e-05, 3.3333e-05, 3.3333e-05, 3.3333e-05])


### 模型
embed → encoder → attend → predict

In [42]:
import torch.nn as nn
import torch.nn.functional as F

In [43]:
class NewsEncoder(nn.Module):
    def __init__(self, embedding_dim, num_word_embeddings, num_char_embeddings,
                 kernels, num_input_channels, num_output_channels, 
                 rnn_hidden_dim, num_layers, bidirectional, 
                 word_padding_idx=0, char_padding_idx=0):
        super(NewsEncoder, self).__init__()
        
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        
        # Embeddings
        self.word_embeddings = nn.Embedding(embedding_dim=embedding_dim,
                                            num_embeddings=num_word_embeddings,
                                            padding_idx=word_padding_idx)
        self.char_embeddings = nn.Embedding(embedding_dim=embedding_dim,
                                            num_embeddings=num_char_embeddings,
                                            padding_idx=char_padding_idx)
        
        # Conv weights
        self.conv = nn.ModuleList([nn.Conv1d(num_input_channels, 
                                             num_output_channels, 
                                             kernel_size=f) for f in kernels])
        
        
        # GRU weights
        self.gru = nn.GRU(input_size=embedding_dim*(len(kernels)+1), 
                          hidden_size=rnn_hidden_dim, num_layers=num_layers, 
                          batch_first=True, bidirectional=bidirectional)
        
    def initialize_hidden_state(self, batch_size, rnn_hidden_dim, device):
        """Modify this to condition the RNN."""
        num_directions = 1
        if self.bidirectional:
            num_directions = 2
        hidden_t = torch.zeros(self.num_layers * num_directions, 
                               batch_size, rnn_hidden_dim).to(device)
        
    def get_char_level_embeddings(self, x):
        # x: (N, seq_len, word_len)
        input_shape = x.size()
        batch_size, seq_len, word_len = input_shape
        x = x.view(-1, word_len) # (N*seq_len, word_len)
        
        # Embedding
        x = self.char_embeddings(x) # (N*seq_len, word_len, embedding_dim)
        
        # Rearrange input so num_input_channels is in dim 1 (N, embedding_dim, word_len)
        x = x.transpose(1, 2)
        
        # Convolution
        z = [F.relu(conv(x)) for conv in self.conv]
        
        # Pooling
        z = [F.max_pool1d(zz, zz.size(2)).squeeze(2) for zz in z] 
        z = [zz.view(batch_size, seq_len, -1) for zz in z] # (N, seq_len, embedding_dim)
        
        # Concat to get char-level embeddings
        z = torch.cat(z, 2) # join conv outputs
        
        return z
        
    def forward(self, x_word, x_char, x_lengths, device):
        """
        x_word: word level representation (N, seq_size)
        x_char: char level representation (N, seq_size, word_len)
        """
        
        # Word level embeddings
        z_word = self.word_embeddings(x_word)
        
        # Char level embeddings
        z_char = self.get_char_level_embeddings(x=x_char)
        
        # Concatenate
        z = torch.cat([z_word, z_char], 2)
        
        # Feed into RNN
        initial_h = self.initialize_hidden_state(
            batch_size=z.size(0), rnn_hidden_dim=self.gru.hidden_size,
            device=device)
        out, h_n = self.gru(z, initial_h)
        
        return out

In [44]:
class NewsDecoder(nn.Module):
    def __init__(self, rnn_hidden_dim, hidden_dim, output_dim, dropout_p):
        super(NewsDecoder, self).__init__()
        
        # 注意力全连接模型
        self.fc_attn = nn.Linear(rnn_hidden_dim, rnn_hidden_dim)
        self.v = nn.Parameter(torch.rand(rnn_hidden_dim))
        
        # 全连接权重
        self.dropout = nn.Dropout(dropout_p)
        self.fc1 = nn.Linear(rnn_hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, encoder_outputs, apply_softmax=False):
        
        # Attention
        z = torch.tanh(self.fc_attn(encoder_outputs))
        z = z.transpose(2,1) # [B*H*T]
        v = self.v.repeat(encoder_outputs.size(0),1).unsqueeze(1) #[B*1*H]
        z = torch.bmm(v,z).squeeze(1) # [B*T]
        attn_scores = F.softmax(z, dim=1)
        context = torch.matmul(encoder_outputs.transpose(-2, -1), 
                               attn_scores.unsqueeze(dim=2)).squeeze()
        if len(context.size()) == 1:
            context = context.unsqueeze(0)
        
        # 全连接层
        z = self.dropout(context)
        z = self.fc1(z)
        z = self.dropout(z)
        y_pred = self.fc2(z)

        if apply_softmax:
            y_pred = F.softmax(y_pred, dim=1)
        return attn_scores, y_pred

In [45]:
class NewsModel(nn.Module):
    def __init__(self, embedding_dim, num_word_embeddings, num_char_embeddings,
                 kernels, num_input_channels, num_output_channels, 
                 rnn_hidden_dim, hidden_dim, output_dim, num_layers, 
                 bidirectional, dropout_p, word_padding_idx, char_padding_idx):
        super(NewsModel, self).__init__()
        self.encoder = NewsEncoder(embedding_dim, num_word_embeddings,
                                   num_char_embeddings, kernels, 
                                   num_input_channels, num_output_channels, 
                                   rnn_hidden_dim, num_layers, bidirectional, 
                                   word_padding_idx, char_padding_idx)
        self.decoder = NewsDecoder(rnn_hidden_dim, hidden_dim, output_dim, 
                                   dropout_p)
        
    def forward(self, x_word, x_char, x_lengths, device, apply_softmax=False):
        encoder_outputs = self.encoder(x_word, x_char, x_lengths, device)
        y_pred = self.decoder(encoder_outputs, apply_softmax)
        return y_pred

### 训练

In [46]:
import torch.optim as optim

In [47]:
class Trainer(object):
    def __init__(self, dataset, model, model_state_file, save_dir, device, 
                 shuffle, num_epochs, batch_size, learning_rate, 
                 early_stopping_criteria):
        self.dataset = dataset
        self.class_weights = dataset.class_weights.to(device)
        self.device = device
        self.model = model.to(device)
        self.save_dir = save_dir
        self.device = device
        self.shuffle = shuffle
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.loss_func = nn.CrossEntropyLoss(self.class_weights)
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer=self.optimizer, mode='min', factor=0.5, patience=1)
        self.train_state = {
            'stop_early': False, 
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'early_stopping_criteria': early_stopping_criteria,
            'learning_rate': learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': model_state_file}
    
    def update_train_state(self):

        # Verbose
        print ("[EPOCH]: {0:02d} | [LR]: {1} | [TRAIN LOSS]: {2:.2f} | [TRAIN ACC]: {3:.1f}% | [VAL LOSS]: {4:.2f} | [VAL ACC]: {5:.1f}%".format(
          self.train_state['epoch_index'], self.train_state['learning_rate'], 
            self.train_state['train_loss'][-1], self.train_state['train_acc'][-1], 
            self.train_state['val_loss'][-1], self.train_state['val_acc'][-1]))

        # Save one model at least
        if self.train_state['epoch_index'] == 0:
            torch.save(self.model.state_dict(), self.train_state['model_filename'])
            self.train_state['stop_early'] = False

        # Save model if performance improved
        elif self.train_state['epoch_index'] >= 1:
            loss_tm1, loss_t = self.train_state['val_loss'][-2:]

            # If loss worsened
            if loss_t >= self.train_state['early_stopping_best_val']:
                # Update step
                self.train_state['early_stopping_step'] += 1

            # Loss decreased
            else:
                # Save the best model
                if loss_t < self.train_state['early_stopping_best_val']:
                    torch.save(self.model.state_dict(), self.train_state['model_filename'])

                # Reset early stopping step
                self.train_state['early_stopping_step'] = 0

            # Stop early ?
            self.train_state['stop_early'] = self.train_state['early_stopping_step'] \
              >= self.train_state['early_stopping_criteria']
        return self.train_state
  
    def compute_accuracy(self, y_pred, y_target):
        _, y_pred_indices = y_pred.max(dim=1)
        n_correct = torch.eq(y_pred_indices, y_target).sum().item()
        return n_correct / len(y_pred_indices) * 100
    
    def pad_word_seq(self, seq, length):
        vector = np.zeros(length, dtype=np.int64)
        vector[:len(seq)] = seq
        vector[len(seq):] = self.dataset.vectorizer.title_word_vocab.mask_index
        return vector
    
    def pad_char_seq(self, seq, seq_length, word_length):
        vector = np.zeros((seq_length, word_length), dtype=np.int64)
        vector.fill(self.dataset.vectorizer.title_char_vocab.mask_index)
        for i in range(len(seq)):
            char_padding = np.zeros(word_length-len(seq[i]), dtype=np.int64)
            vector[i] = np.concatenate((seq[i], char_padding), axis=None)
        return vector
        
    def collate_fn(self, batch):
        
        # Make a deep copy
        batch_copy = copy.deepcopy(batch)
        processed_batch = {"title_word_vector": [], "title_char_vector": [], 
                           "title_length": [], "category": []}
             
        # Max lengths
        get_seq_length = lambda sample: len(sample["title_word_vector"])
        get_word_length = lambda sample: len(sample["title_char_vector"][0])
        max_seq_length = max(map(get_seq_length, batch))
        max_word_length = max(map(get_word_length, batch))


        # Pad
        for i, sample in enumerate(batch_copy):
            padded_word_seq = self.pad_word_seq(
                sample["title_word_vector"], max_seq_length)
            padded_char_seq = self.pad_char_seq(
                sample["title_char_vector"], max_seq_length, max_word_length)
            processed_batch["title_word_vector"].append(padded_word_seq)
            processed_batch["title_char_vector"].append(padded_char_seq)
            processed_batch["title_length"].append(sample["title_length"])
            processed_batch["category"].append(sample["category"])
            
        # Convert to appropriate tensor types
        processed_batch["title_word_vector"] = torch.LongTensor(
            processed_batch["title_word_vector"])
        processed_batch["title_char_vector"] = torch.LongTensor(
            processed_batch["title_char_vector"])
        processed_batch["title_length"] = torch.LongTensor(
            processed_batch["title_length"])
        processed_batch["category"] = torch.LongTensor(
            processed_batch["category"])
        
        return processed_batch  
  
    def run_train_loop(self):
        for epoch_index in range(self.num_epochs):
            self.train_state['epoch_index'] = epoch_index
      
            # Iterate over train dataset

            # initialize batch generator, set loss and acc to 0, set train mode on
            self.dataset.set_split('train')
            batch_generator = self.dataset.generate_batches(
                batch_size=self.batch_size, collate_fn=self.collate_fn, 
                shuffle=self.shuffle, device=self.device)
            running_loss = 0.0
            running_acc = 0.0
            self.model.train()

            for batch_index, batch_dict in enumerate(batch_generator):
                # zero the gradients
                self.optimizer.zero_grad()
                
                # compute the output
                _, y_pred = self.model(x_word=batch_dict['title_word_vector'],
                                       x_char=batch_dict['title_char_vector'],
                                       x_lengths=batch_dict['title_length'],
                                       device=self.device)
                
                # compute the loss
                loss = self.loss_func(y_pred, batch_dict['category'])
                loss_t = loss.item()
                running_loss += (loss_t - running_loss) / (batch_index + 1)

                # compute gradients using loss
                loss.backward()

                # use optimizer to take a gradient step
                self.optimizer.step()
                
                # compute the accuracy
                acc_t = self.compute_accuracy(y_pred, batch_dict['category'])
                running_acc += (acc_t - running_acc) / (batch_index + 1)

            self.train_state['train_loss'].append(running_loss)
            self.train_state['train_acc'].append(running_acc)

            # Iterate over val dataset

            # initialize batch generator, set loss and acc to 0, set eval mode on
            self.dataset.set_split('val')
            batch_generator = self.dataset.generate_batches(
                batch_size=self.batch_size, collate_fn=self.collate_fn, 
                shuffle=self.shuffle, device=self.device)
            running_loss = 0.
            running_acc = 0.
            self.model.eval()

            for batch_index, batch_dict in enumerate(batch_generator):

                # compute the output
                _, y_pred = self.model(x_word=batch_dict['title_word_vector'],
                                       x_char=batch_dict['title_char_vector'],
                                       x_lengths=batch_dict['title_length'],
                                       device=self.device)

                # compute the loss
                loss = self.loss_func(y_pred, batch_dict['category'])
                loss_t = loss.to("cpu").item()
                running_loss += (loss_t - running_loss) / (batch_index + 1)

                # compute the accuracy
                acc_t = self.compute_accuracy(y_pred, batch_dict['category'])
                running_acc += (acc_t - running_acc) / (batch_index + 1)

            self.train_state['val_loss'].append(running_loss)
            self.train_state['val_acc'].append(running_acc)

            self.train_state = self.update_train_state()
            self.scheduler.step(self.train_state['val_loss'][-1])
            if self.train_state['stop_early']:
                break
          
    def run_test_loop(self):
        # initialize batch generator, set loss and acc to 0, set eval mode on
        self.dataset.set_split('test')
        batch_generator = self.dataset.generate_batches(
            batch_size=self.batch_size, collate_fn=self.collate_fn, 
            shuffle=self.shuffle, device=self.device)
        running_loss = 0.0
        running_acc = 0.0
        self.model.eval()

        for batch_index, batch_dict in enumerate(batch_generator):
            # compute the output
            _, y_pred = self.model(x_word=batch_dict['title_word_vector'],
                                   x_char=batch_dict['title_char_vector'],
                                   x_lengths=batch_dict['title_length'],
                                   device=self.device)

            # compute the loss
            loss = self.loss_func(y_pred, batch_dict['category'])
            loss_t = loss.item()
            running_loss += (loss_t - running_loss) / (batch_index + 1)

            # compute the accuracy
            acc_t = self.compute_accuracy(y_pred, batch_dict['category'])
            running_acc += (acc_t - running_acc) / (batch_index + 1)

        self.train_state['test_loss'] = running_loss
        self.train_state['test_acc'] = running_acc
    
    def plot_performance(self):
        # Figure size
        plt.figure(figsize=(15,5))

        # Plot Loss
        plt.subplot(1, 2, 1)
        plt.title("Loss")
        plt.plot(trainer.train_state["train_loss"], label="train")
        plt.plot(trainer.train_state["val_loss"], label="val")
        plt.legend(loc='upper right')

        # Plot Accuracy
        plt.subplot(1, 2, 2)
        plt.title("Accuracy")
        plt.plot(trainer.train_state["train_acc"], label="train")
        plt.plot(trainer.train_state["val_acc"], label="val")
        plt.legend(loc='lower right')

        # Save figure
        plt.savefig(os.path.join(self.save_dir, "performance.png"))

        # Show plots
        plt.show()
    
    def save_train_state(self):
        with open(os.path.join(self.save_dir, "train_state.json"), "w") as fp:
            json.dump(self.train_state, fp)

In [48]:
# Initialization
dataset = NewsDataset.load_dataset_and_make_vectorizer(args.split_data_file,
                                                       args.cutoff)
dataset.save_vectorizer(args.vectorizer_file)
vectorizer = dataset.vectorizer
model = NewsModel(embedding_dim=args.embedding_dim, 
                  num_word_embeddings=len(vectorizer.title_word_vocab), 
                  num_char_embeddings=len(vectorizer.title_char_vocab),
                  kernels=args.kernels,
                  num_input_channels=args.embedding_dim,
                  num_output_channels=args.num_filters,
                  rnn_hidden_dim=args.rnn_hidden_dim,
                  hidden_dim=args.hidden_dim,
                  output_dim=len(vectorizer.category_vocab),
                  num_layers=args.num_layers,
                  bidirectional=args.bidirectional,
                  dropout_p=args.dropout_p, 
                  word_padding_idx=vectorizer.title_word_vocab.mask_index,
                  char_padding_idx=vectorizer.title_char_vocab.mask_index)
print (model.named_modules)

<bound method Module.named_modules of NewsModel(
  (encoder): NewsEncoder(
    (word_embeddings): Embedding(3406, 100, padding_idx=0)
    (char_embeddings): Embedding(35, 100, padding_idx=0)
    (conv): ModuleList(
      (0): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
      (1): Conv1d(100, 100, kernel_size=(5,), stride=(1,))
    )
    (gru): GRU(300, 128, batch_first=True)
  )
  (decoder): NewsDecoder(
    (fc_attn): Linear(in_features=128, out_features=128, bias=True)
    (dropout): Dropout(p=0.25)
    (fc1): Linear(in_features=128, out_features=200, bias=True)
    (fc2): Linear(in_features=200, out_features=4, bias=True)
  )
)>


In [49]:
# Train
trainer = Trainer(dataset=dataset, model=model, 
                  model_state_file=args.model_state_file, 
                  save_dir=args.save_dir, device=args.device,
                  shuffle=args.shuffle, num_epochs=args.num_epochs, 
                  batch_size=args.batch_size, learning_rate=args.learning_rate, 
                  early_stopping_criteria=args.early_stopping_criteria)
trainer.run_train_loop()

[EPOCH]: 00 | [LR]: 0.001 | [TRAIN LOSS]: 0.78 | [TRAIN ACC]: 69.2% | [VAL LOSS]: 0.54 | [VAL ACC]: 80.1%
[EPOCH]: 01 | [LR]: 0.001 | [TRAIN LOSS]: 0.50 | [TRAIN ACC]: 82.2% | [VAL LOSS]: 0.50 | [VAL ACC]: 82.0%
[EPOCH]: 02 | [LR]: 0.001 | [TRAIN LOSS]: 0.43 | [TRAIN ACC]: 84.6% | [VAL LOSS]: 0.46 | [VAL ACC]: 83.2%
[EPOCH]: 03 | [LR]: 0.001 | [TRAIN LOSS]: 0.39 | [TRAIN ACC]: 86.2% | [VAL LOSS]: 0.46 | [VAL ACC]: 83.7%
[EPOCH]: 04 | [LR]: 0.001 | [TRAIN LOSS]: 0.35 | [TRAIN ACC]: 87.5% | [VAL LOSS]: 0.44 | [VAL ACC]: 84.3%


In [50]:
# Plot performance
trainer.plot_performance()

<Figure size 1080x360 with 2 Axes>

In [51]:
# Test performance
trainer.run_test_loop()
print("Test loss: {0:.2f}".format(trainer.train_state['test_loss']))
print("Test Accuracy: {0:.1f}%".format(trainer.train_state['test_acc']))

Test loss: 0.44
Test Accuracy: 84.6%


In [52]:
# Save all results
trainer.save_train_state()

### 预测

In [53]:
class Inference(object):
    def __init__(self, model, vectorizer):
        self.model = model
        self.vectorizer = vectorizer
  
    def predict_category(self, title):
        # Vectorize
        word_vector, char_vector, title_length = self.vectorizer.vectorize(title)
        title_word_vector = torch.tensor(word_vector).unsqueeze(0)
        title_char_vector = torch.tensor(char_vector).unsqueeze(0)
        title_length = torch.tensor([title_length]).long()        
        
        # Forward pass
        self.model.eval()
        attn_scores, y_pred = self.model(x_word=title_word_vector, 
                                         x_char=title_char_vector,
                                         x_lengths=title_length, 
                                         device="cpu",
                                         apply_softmax=True)

        # Top category
        y_prob, indices = y_pred.max(dim=1)
        index = indices.item()

        # Predicted category
        category = vectorizer.category_vocab.lookup_index(index)
        probability = y_prob.item()
        return {'category': category, 'probability': probability, 
                'attn_scores': attn_scores}
    
    def predict_top_k(self, title, k):
        # Vectorize
        word_vector, char_vector, title_length = self.vectorizer.vectorize(title)
        title_word_vector = torch.tensor(word_vector).unsqueeze(0)
        title_char_vector = torch.tensor(char_vector).unsqueeze(0)
        title_length = torch.tensor([title_length]).long()
        
         # Forward pass
        self.model.eval()
        _, y_pred = self.model(x_word=title_word_vector,
                               x_char=title_char_vector,
                               x_lengths=title_length, 
                               device="cpu",
                               apply_softmax=True)
        
        # Top k categories
        y_prob, indices = torch.topk(y_pred, k=k)
        probabilities = y_prob.detach().numpy()[0]
        indices = indices.detach().numpy()[0]

        # Results
        results = []
        for probability, index in zip(probabilities, indices):
            category = self.vectorizer.category_vocab.lookup_index(index)
            results.append({'category': category, 'probability': probability})

        return results

In [54]:
# Load the model
dataset = NewsDataset.load_dataset_and_load_vectorizer(
    args.split_data_file, args.vectorizer_file)
vectorizer = dataset.vectorizer
model = NewsModel(embedding_dim=args.embedding_dim, 
                  num_word_embeddings=len(vectorizer.title_word_vocab), 
                  num_char_embeddings=len(vectorizer.title_char_vocab),
                  kernels=args.kernels,
                  num_input_channels=args.embedding_dim,
                  num_output_channels=args.num_filters,
                  rnn_hidden_dim=args.rnn_hidden_dim,
                  hidden_dim=args.hidden_dim,
                  output_dim=len(vectorizer.category_vocab),
                  num_layers=args.num_layers,
                  bidirectional=args.bidirectional,
                  dropout_p=args.dropout_p, 
                  word_padding_idx=vectorizer.title_word_vocab.mask_index,
                  char_padding_idx=vectorizer.title_char_vocab.mask_index)
model.load_state_dict(torch.load(args.model_state_file))
model = model.to("cpu")
print (model.named_modules)

<bound method Module.named_modules of NewsModel(
  (encoder): NewsEncoder(
    (word_embeddings): Embedding(3406, 100, padding_idx=0)
    (char_embeddings): Embedding(35, 100, padding_idx=0)
    (conv): ModuleList(
      (0): Conv1d(100, 100, kernel_size=(3,), stride=(1,))
      (1): Conv1d(100, 100, kernel_size=(5,), stride=(1,))
    )
    (gru): GRU(300, 128, batch_first=True)
  )
  (decoder): NewsDecoder(
    (fc_attn): Linear(in_features=128, out_features=128, bias=True)
    (dropout): Dropout(p=0.25)
    (fc1): Linear(in_features=128, out_features=200, bias=True)
    (fc2): Linear(in_features=200, out_features=4, bias=True)
  )
)>


In [55]:
# Inference
inference = Inference(model=model, vectorizer=vectorizer)
title = input("Enter a title to classify: ")
prediction = inference.predict_category(preprocess_text(title))
print("{} → {} (p={:0.2f})".format(title, prediction['category'], 
                                   prediction['probability']))

Enter a title to classify: : shanghai daily

shanghai daily → Sports (p=0.51)


In [56]:
# Top-k inference
top_k = inference.predict_top_k(preprocess_text(title), k=len(vectorizer.category_vocab))
print ("{}: ".format(title))
for result in top_k:
    print ("{} (p={:0.2f})".format(result['category'], 
                                   result['probability']))

shanghai daily: 
Sports (p=0.51)
World (p=0.25)
Business (p=0.19)
Sci/Tech (p=0.04)


## 可解释性


我们可查看每个时间步生成的概率向量，然后对   前面每一个隐藏状态对某一时间步的预测结果的影响  做可视化

In [57]:
import seaborn as sns
import matplotlib.pyplot as plt

In [59]:
attn_matrix = prediction['attn_scores'].detach().numpy()
ax = sns.heatmap(attn_matrix, linewidths=2, square=True)
tokens = ["<BEGIN>"]+preprocess_text(title).split(" ")+["<END>"]
ax.set_xticklabels(tokens, rotation=45)
ax.set_xlabel("Token")
ax.set_ylabel("Importance\n")
plt.show()

<Figure size 432x288 with 2 Axes>